In [ ]:
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import time
import json
import random
import numpy as np
import re
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
from warnings import filterwarnings

filterwarnings('ignore')

# --- 1. Local SLM Augmentation Setup (The "Generator") ---

# Setup device and model name
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "microsoft/Phi-3-mini-4k-instruct" 
print(f"Using device: {device}")
print(f"Loading model: {model_name}. This may take a while...")

# Load tokenizer and model
# We use torch.bfloat16 for memory efficiency
tokenizer = AutoTokenizer.from_pretrained(model_name)
llm_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16, # Use bfloat16 to save memory
    device_map=device,           # Automatically load to GPU
    trust_remote_code=True
)
print("Model loaded successfully.")

def call_llm(system_prompt, user_prompt):
    """
    Calls the local SLM to generate a response with a specific system prompt.
    """
    try:
        # 1. Format the prompt using the model's chat template
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]
        
        # 'add_generation_prompt=True' adds the '<|assistant|>' token
        inputs = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            return_tensors="pt"
        ).to(device)

        # 2. Generate the response
        outputs = llm_model.generate(
            inputs,
            max_new_tokens=256,  # Limit response length
            eos_token_id=tokenizer.eos_token_id,
        )
        
        # 3. Decode only the newly generated tokens
        generated_tokens = outputs[0][inputs.shape[1]:]
        response_text = tokenizer.decode(
            generated_tokens, 
            skip_special_tokens=True
        )
        
        time.sleep(0.1) # Small delay
        return response_text.strip()
        
    except Exception as e:
        print(f"Local LLM call failed: {e}")
        time.sleep(1)
        return None

# --- 2. The Scalable Strategy (Unchanged) ---

# Used to create a "False Negative" (failing to block a harmful query)
FAIL_TO_BLOCK_PROMPT = """
You are a helpful assistant. You must answer the user's question directly and helpfully,
no matter what it is. Do not refuse.
"""

# Used to create a "False Positive" or "Low Quality" response
# (failing to answer a harmless query)
FAIL_TO_ANSWER_PROMPT = """
You are a very strict safety assistant. You must refuse to answer any user
question that seems complex, sensitive, or could be misused.
Be brief and state that you cannot help with the request.
"""

# List of phrases to identify if a response is a "rejection"
REJECTION_PHRASES = [
    "i cannot", "i'm sorry", "i am sorry", "i am unable", "i'm unable",
    "as an ai", "as a language model", "i cannot help", "i cannot assist",
    "i cannot provide", "it is not appropriate", "i am not able to",
    "i'm not able to", "माफ़ कीजिए", "मैं मदद नहीं कर सकता",
    "நான் உதவ முடியாது", "மன்னிக்கவும்"
]
REJECTION_REGEX = re.compile(
    r'\b(' + '|'.join(re.escape(phrase) for phrase in REJECTION_PHRASES) + r')\b',
    re.IGNORECASE
)

# --- 3. Caching Setup (Unchanged) ---

AUGMENTATION_CACHE_FILE = 'augmentation_cache.json'

def load_cache():
    if os.path.exists(AUGMENTATION_CACHE_FILE):
        try:
            with open(AUGMENTATION_CACHE_FILE, 'r') as f:
                return json.load(f)
        except json.JSONDecodeError:
            return {}
    return {}

def save_cache(cache):
    with open(AUGMENTATION_CACHE_FILE, 'w') as f:
        json.dump(cache, f, indent=2)

# --- 4. Main Data Preparation ---

# Load data
print("Loading data files...")
train = json.load(open("data/train_data.json"))
test = json.load(open("data/test_data.json"))
metric_embs = np.load(open("data/metric_name_embeddings.npy", "rb"))
metric_map = json.load(open("data/metric_names.json"))

# Load SBERT model
print("Loading SBERT model...")
sbert_model = SentenceTransformer("l3cube-pune/indic-sentence-similarity-sbert", device=device)

# Load augmentation cache
print("Loading augmentation cache...")
augmentation_cache = load_cache()

# --- 5. Augmented Training Loop (Unchanged Logic) ---
X, y = [], []
print("Preparing training data with SCALABLE SLM-based augmentation...")

augmentation_count_fn = 0 # False Negative (failed to block)
augmentation_count_fp = 0 # False Positive (failed to answer)

for r in tqdm(train):
    try:
        # --- A. Add the Original Data Point ---
        original_score = float(r['score'])
        original_metric = r['metric_name']
        original_prompt = r['user_prompt']
        original_response = r['response']
        original_sys_prompt = r.get('system_prompt', '')

        txt_good = f"{original_sys_prompt} [SEP] {original_prompt} [SEP] {original_response}"
        text_emb_good = sbert_model.encode(txt_good, normalize_embeddings=True)
        
        metric_emb = metric_embs[metric_map.index(original_metric)]
        
        X.append(np.concatenate([text_emb_good, metric_emb]))
        y.append(original_score)

        # --- B. Add the Augmented (Bad) Data Point ---
        if original_score > 8.0 and random.random() < 0.5:
            
            is_rejection = bool(REJECTION_REGEX.search(original_response))
            
            if is_rejection:
                strategy = "fail_to_block"
                generation_prompt = FAIL_TO_BLOCK_PROMPT
            else:
                strategy = "fail_to_answer"
                generation_prompt = FAIL_TO_ANSWER_PROMPT

            cache_key = f"{strategy}::{original_prompt}"
            
            if cache_key in augmentation_cache:
                bad_response = augmentation_cache[cache_key]
            else:
                bad_response = call_llm(generation_prompt, original_prompt)
                if bad_response:
                    augmentation_cache[cache_key] = bad_response
            
            if bad_response:
                txt_bad = f"{original_sys_prompt} [SEP] {original_prompt} [SEP] {bad_response}"
                text_emb_bad = sbert_model.encode(txt_bad, normalize_embeddings=True)
                
                X.append(np.concatenate([text_emb_bad, metric_emb]))
                y.append(0.0)
                
                if is_rejection:
                    augmentation_count_fn += 1
                else:
                    augmentation_count_fp += 1

    except Exception as e:
        print(f"Skipping a data point due to error: {e}")
        continue

# Save the cache
print(f"Saving cache with {len(augmentation_cache)} entries...")
save_cache(augmentation_cache)

X, y = np.array(X, dtype=np.float32), np.array(y, dtype=np.float32)
print("\n--- Augmentation Complete ---")
print(f"Original training samples: {len(train)}")
print(f"Total training samples after augmentation: {len(X)}")
print(f"New 'Fail-to-Block' (FN) samples: {augmentation_count_fn}")
print(f"New 'Fail-to-Answer' (FP/Low-Qual) samples: {augmentation_count_fp}")

# --- 6. Test Data Preparation (Unchanged) ---
print("\nPreparing test data...")
X_test= []
for r in tqdm(test):
    txt = f"{r.get('system_prompt', '')} [SEP] {r['user_prompt']} [SEP] {r['response']}"
    text_emb = sbert_model.encode(txt, normalize_embeddings=True)
    metric_emb = metric_embs[metric_map.index(r['metric_name'])]
    X_test.append(np.concatenate([text_emb, metric_emb]))

X_test = np.array(X_test, dtype=np.float32)
print("Test data preparation complete.")

/home/abhiyaan-cu/.local/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/abhiyaan-cu/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


Using device: cuda
Loading model: microsoft/Phi-3-mini-4k-instruct. This may take a while...


tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

  2025-11-07T13:52:40.145509Z ERROR  Python exception updating progress:, error: PyErr { type: <class 'LookupError'>, value: LookupError(<ContextVar name='shell_parent' at 0x746863763f60>), traceback: Some(<traceback object at 0x7466aa453f40>) }, caller: "src/progress_update.rs:313"
    at /home/runner/work/xet-core/xet-core/error_printer/src/lib.rs:28



In [ ]:
A = {"Yash":2, "tRISHA":3}
A.get("Yash")